# Causal profit

##### Datasets

1. Hillstrom
2. Bank
3. Criteo
4. Synthetic

##### Causal models

1. T_learner
2. S_learner

##### Algorithms

1. Logistic regression (LR)
2. XGBoost (XGB)

In [1]:
pip install -r requirements.txt

  Using cached CausalInference-0.1.2.tar.gz (58 kB)
  Using cached causallift-0.0.3-py3-none-any.whl (12 kB)
  Using cached causalml-0.8.0-cp37-cp37m-macosx_10_9_x86_64.whl (213 kB)
  Using cached econml-0.7.0-py3-none-any.whl (312 kB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
  Using cached matplotlib-3.3.4-cp37-cp37m-macosx_10_9_x86_64.whl (8.5 MB)
  Using cached numpy-1.19.1-cp37-cp37m-macosx_10_9_x86_64.whl (15.3 MB)
  Using cached pandas-1.1.2-cp37-cp37m-macosx_10_9_x86_64.whl (10.4 MB)
  Using cached plotly-4.11.0-py2.py3-none-any.whl (13.1 MB)
  Using cached scikit_learn-0.23.2-cp37-cp37m-macosx_10_9_x86_64.whl (7.2 MB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached scikit_uplift-0.2.0-py3-none-any.whl (20 kB)
  Using cached scipy-1.4.1-cp37-cp37m-macosx_10_6_intel.whl (28.4 MB)
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
     |████████████████████████████████| 1.3 MB 1.3 MB/s eta 0:00:01
     |██████████████████████████████

In [ ]:
from _init_ import *
root = os.getcwd()

### 1. Load data

In [ ]:
'''
load_data
> Parameters: name_dataset, path datasets
> Returns: dictionary containing: 
    1. 'name': Name dataset
    2. 'folds': Folds for training
    3. 'T': Name treatment
    4. 'X': Names features
    5. 'Y': Name target
    6. 'descriptive': Descriptives
'''
path_datasets = root + 'datasets/' 
data = load_data('Synthetic',path_datasets)

### 2. Train causal model

1. Train causal models: 'load_model' = False
2. Load pretrained causal models: 'load_model' = True

In [ ]:
'''
train_causal
> Parameters: 
    1. Path to directory where models are saved
    2. Name of dataset
    3. Folds
    4. Name treatment
    5. Name target
    6. Name features
    7. Algorithm 
    8. Load_model == True if model was saved
> Returns: list of dfs with predictions
'''
path_models = root + '/models/'
effect_estimates = train_causal(path_models, data['name'],data['folds'],data['T'],data['Y'],data['X'],'XGB',True)

### 3. Performance metrics

In [ ]:
"""
Specify the b and c parameters:
"""
#b = [10,100,200]
b = [100]
#benefit = 10
#c_pro = [0.2,0.3,0.4]
c = 0.1

'''
performance_metrics
> Parameters: 
    1. Predictions: effect_estimates
    2. Name of causal model: [CF,T_learner,S_learner]
    3. Name treatment
    4. Name target
    5. Benefit parameters: b
    6. Cost parameters: c
> Returns: list of performance metrics and data for plots
'''

performance = performance_metrics(effect_estimates,'T_learner',data['T'],data['Y'],b,c)
performance

### 4. Plots
#### Qini

In [ ]:
ax = sns.lineplot(x='x', y= 'Value', hue='Approach', style = 'Approach',lw=1,data = performance['plot_qinis'])
plt.xlabel("Targeted proportion")
plt.ylabel("Cumulative Qini")
#plt.xticks(np.linspace(0, 100, 11, endpoint = True))
#ax.legend(loc = 4, frameon = False, ncol = 1, labelspacing= 0.2)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.legend(loc=4, borderaxespad=0.1)
#fig_uplift = ax.get_figure()
#fig_uplift.savefig('Qini_Hillstrom.pdf',bbox_inches='tight',transparent =True)

#### Profit

In [ ]:
sns.set_style("white")
flatui = ["#1f77b4", "#2ca02c"]
ax = sns.lineplot(x='x', y= 'Value', hue='Approach', palette=flatui,lw=1,data = performance['plot_profit'])
plt.xlabel("Targeted proportion")
plt.ylabel("Profit per instance")
#plt.xticks(np.linspace(0, 100, 11, endpoint = True))
#ax.legend(loc = 4, frameon = False, ncol = 1, labelspacing= 0.2)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.legend(loc=2, borderaxespad=0.1)#loc = 4
#fig_uplift = ax.get_figure()
#fig_uplift.savefig('b10_CF_Criteo.pdf',bbox_inches='tight',transparent =True)

#### Cumulative positive instances in T and C

In [ ]:
sns.set_style("white")
ax = sns.lineplot(x='x', y= 'Value', hue='Approach', style = 'Approach',lw=1,data = performance['plot_cum_t'])